# Importing the necessary modules

In [14]:
# Utilitaries
from csv import reader
import re
import pandas as pd
import zeep
# Text manipulation

# Data preparation

# Machine learning tools
from sklearn.model_selection import train_test_split
# Machine learning classificators

# Reading the corpus

In [2]:
df = pd.read_csv('corpus.csv')

In [3]:
df.sample(5)

,author,title,translation,label
928,TVR,"Rezultate parțiale BEC, după centralizarea a 9...","PSD a obținut la alegerile parlamentare 29,64...",1
920,TVR,Marcel Ciolacu: Voi avea o discuție cu colegii...,"Președintele PSD, Marcel Ciolacu, a declarat c...",1
430,Ionut,Cel mai norocos oltean. A găsit o sarma cu pat...,"Chiar dacă a găsit sarmaua cu patru foi, oltea...",0
700,TVR,"Bugetul, dezbătut intens în comisiile de speci...",ACTUALIZARE Bugetul Casei Naţionale de Asigură...,1
342,Eftimie,Fum alb la negocieri. Ludovic Orban fumează ia...,"„Doar un fum să mai trag”, a declarat liderul ...",0


# Data preparation

### Lemmatization
_Source: http://nlptools.info.uaic.ro/WebPosRo/_

In [4]:
wsdl = 'http://nlptools.info.uaic.ro/WebPosRo/PosTaggerRoWS?wsdl'

In [5]:
client = zeep.Client(wsdl=wsdl)

### Transforming each article using the POS tagger

In [ ]:
lemmatized_articles_xml=[]

In [6]:
# for article in df['translation']:
#      lemmatized_articles_xml.append(client.service.parseSentence_XML(article))
    

In [10]:
# # Saving the articles to save execution time
# for article in lemmatized_articles_xml:
#     with open('lemmatized_articles','a+') as f:
#         f.write(article)
#         f.write('\n')

In [17]:
for article in lemmatized_articles_xml:
    arr = article.split('\n')
    arr_lemmas = []
    final_tokenized = []
    index = 0
    for line in arr:
        found = re.search('.*(LEMMA="[A-Za-zăâîșț1-9]*").*', line)
        if found:
            res = found.group(1)
            arr_lemmas.append(res)
    for lemma in arr_lemmas:
        found = re.search('LEMMA="([A-Za-zăâîșț1-9]*)".*', lemma)
        if found:
            res = found.group(1)
            final_tokenized.append(res)
            index = index + 1
            break
    print(final_tokenized)
    break

['dar']
